Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
`https://github.com/squillero/computational-intelligence`  
Free for personal or classroom use; see 'LICENCE.md' for details.

In [3]:
import logging
from collections import deque
from pprint import pprint
import numpy as np

logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%H:%M:%S', level=logging.INFO)

In [4]:
def _contains_duplicates(X):
    return np.sum(np.unique(X)) != np.sum(X)

def contains_duplicates(sol):
    return any(_contains_duplicates(sol[r,:]) for r in range(9)) or \
           any(_contains_duplicates(sol[:,r]) for r in range(9)) or \
           any(_contains_duplicates(sol[r:r+3:,c:c+3]) for r in range(0,9,3) for c in range(0,9,3))

def valid_solution(sol):
    return not contains_duplicates(sol) and np.sum(sol) == (1+2+3+4+5+6+7+8+9) * 9

def print_sudoku(sudoku):
    print("+-------+-------+-------+")
    for b in range(0, 9, 3):
        for r in range(3):
            print("|", " | ".join(" ".join(str(_) for _ in sudoku[b+r, c:c+3]) for c in range(0, 9, 3)), "|")
        print("+-------+-------+-------+")

In [5]:
def dfsolve(sudoku):
    """Vanilla depth-first solver for sudoku puzzles"""
    frontier = deque([sudoku.copy()])
    num_nodes = 0
    while frontier:
        node = frontier.popleft()
        num_nodes += 1

        if valid_solution(node):
            logging.info(f"Solved after expanding {num_nodes:,} nodes")
            return node

        for i, j in zip(*np.where(node == 0)):
            for c in range(1, 10):
                node[i, j] = c
                if not contains_duplicates(node):
                    frontier.appendleft(node.copy())
    logging.info(f"Giving up after expanding {num_nodes:,} nodes")
    return None

In [6]:
simple_sudoku = np.array([[6, 0, 4,    0, 7, 0,    0, 0, 1],
                          [0, 5, 0,    0, 0, 0,    0, 7, 0], 
                          [7, 0, 0,    5, 9, 6,    8, 3, 4], 
       
                          [0, 8, 0,    0, 0, 2,    4, 9, 0], 
                          [1, 0, 0,    0, 0, 0,    0, 0, 3], 
                          [0, 6, 9,    7, 0, 0,    0, 5, 0], 
       
                          [9, 1, 8,    3, 6, 7,    0, 0, 5], 
                          [0, 4, 0,    0, 0, 0,    0, 6, 0], 
                          [2, 0, 0,    0, 5, 0,    7, 0, 8]], dtype=np.int8)

In [7]:
def sudoku_generator(sudokus=1, *, kappa=5, random_seed=None):
    if random_seed:
        np.random.seed(random_seed)
    for puzzle in range(sudokus):
        sudoku = np.zeros((9, 9), dtype=np.int8)
        for cell in range(np.random.randint(kappa)):
            for p, val in zip(np.random.randint(0, 8, size=(9, 2)), range(1, 10)):
                tmp = sudoku.copy()
                sudoku[tuple(p)] = val
                if contains_duplicates(sudoku):
                    sudoku = tmp
        yield sudoku.copy()

In [8]:
def my_sudoku_solver(sudoku):
    ''' 
    Author: @AleVersace (Alessandro Versace)

    Computational Intelligence Lab1 @ Polytechnic University of Turin (PoliTo)

    - Write a program that solves a sudoku in an efficient way 

    Approach: I decided to use a Depth-First Search starting solving the problem
    selecting the "most interesting" empty cell (0s) to change at each step.
    The "Most Interesting" is referred to the cell that has the lowest number 
    of available values that are good (not against the rules) at a certain step. 
    (As we humans usually proceed with the game)

    So at each step the algorithm calculates the available values for each empty cell and selects
    the cell with the lowest number of values. Then it expands the frontier using DF and proceeding
    with the next step.

    '''

    frontier = deque()
    frontier.append(sudoku.copy())

    n = 0
    while frontier:
        n += 1
        node = frontier.popleft()
        
        ## DEBUG
        #print(sudoku1)
        #print(len(frontier))
        ## END

        if valid_solution(node):
            return n, node

        b_i = -1;
        b_j = -1;
        best_available_values = None

        # Check for next interesting position to go on with
        for i, j in np.ndindex(node.shape):
            if node[i, j] == 0:
                # Computes available values that follows game rules for the empty cells
                available_values = np.linspace(1, 9, 9, dtype=np.int8)
                already_used = np.unique(np.concatenate((node[i, :], node[:, j], node[ (i//3*3) : (i//3*3+3) , (j//3*3) : (j//3*3+3)].flatten())))
                available_values = set(available_values.flatten())
                already_used = set(already_used.flatten())
                available_values = np.array(list(available_values.difference(already_used)), dtype=np.int8)
                
                # Save position and values if is the lowest number of available values seen for this configuration
                if best_available_values is None or (available_values.shape[0] < best_available_values.shape[0]):
                    best_available_values = available_values
                    b_i = i
                    b_j = j

        if best_available_values is None:
            continue

        # Expand Frontier
        for v in best_available_values:

            ## DEBUG
            #print(f"{best_available_values} in {b_i},{b_j}\n")
            #print(f"[v] = {v}")
            ## END

            node[b_i, b_j] = v
            new_config = np.copy(node)
            frontier.appendleft(new_config)

In [9]:
for sudoku in sudoku_generator(random_seed=42):
    print_sudoku(sudoku)
    solution = dfsolve(sudoku)
    if solution is not None:
        print_sudoku(solution)

+-------+-------+-------+
| 0 0 1 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 3 0 |
| 0 0 6 | 8 0 0 | 5 2 0 |
+-------+-------+-------+
| 9 7 0 | 4 0 0 | 0 8 0 |
| 6 0 0 | 0 3 0 | 1 0 0 |
| 0 0 0 | 0 8 6 | 0 0 0 |
+-------+-------+-------+
| 0 4 0 | 9 0 0 | 0 0 0 |
| 0 0 9 | 5 7 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+


[16:40:25] INFO: Solved after expanding 1,735 nodes


+-------+-------+-------+
| 8 9 1 | 3 5 2 | 7 6 4 |
| 7 5 2 | 6 4 1 | 9 3 8 |
| 4 3 6 | 8 9 7 | 5 2 1 |
+-------+-------+-------+
| 9 7 3 | 4 1 5 | 6 8 2 |
| 6 8 5 | 2 3 9 | 1 4 7 |
| 2 1 4 | 7 8 6 | 3 9 5 |
+-------+-------+-------+
| 1 4 7 | 9 6 8 | 2 5 3 |
| 3 2 9 | 5 7 4 | 8 1 6 |
| 5 6 8 | 1 2 3 | 4 7 9 |
+-------+-------+-------+


In [10]:
# This is my solution. I separate it to check how fast is the computation.
for sudoku in sudoku_generator(random_seed=42):
    print_sudoku(sudoku)
    
    n, faster_solution = my_sudoku_solver(sudoku)
    print(f"Whoa! Look how good I am, you can't beat me!\nSolved in {n} steps:")
    print_sudoku(faster_solution)

+-------+-------+-------+
| 0 0 1 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 3 0 |
| 0 0 6 | 8 0 0 | 5 2 0 |
+-------+-------+-------+
| 9 7 0 | 4 0 0 | 0 8 0 |
| 6 0 0 | 0 3 0 | 1 0 0 |
| 0 0 0 | 0 8 6 | 0 0 0 |
+-------+-------+-------+
| 0 4 0 | 9 0 0 | 0 0 0 |
| 0 0 9 | 5 7 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+
Whoa! Look how good I am, you can't beat me!
Solved in 67 steps:
+-------+-------+-------+
| 2 9 1 | 6 5 3 | 4 7 8 |
| 8 5 4 | 1 2 7 | 9 3 6 |
| 7 3 6 | 8 9 4 | 5 2 1 |
+-------+-------+-------+
| 9 7 3 | 4 1 5 | 6 8 2 |
| 6 2 8 | 7 3 9 | 1 5 4 |
| 4 1 5 | 2 8 6 | 3 9 7 |
+-------+-------+-------+
| 3 4 7 | 9 6 2 | 8 1 5 |
| 1 6 9 | 5 7 8 | 2 4 3 |
| 5 8 2 | 3 4 1 | 7 6 9 |
+-------+-------+-------+
